In [25]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)



import os
for dirname, _, filenames in os.walk('/kaggle/working'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



/kaggle/working/__notebook_source__.ipynb
/kaggle/working/work/testing/Degas/213309.jpg
/kaggle/working/work/testing/Degas/213454.jpg
/kaggle/working/work/testing/Degas/213756.jpg
/kaggle/working/work/testing/Degas/213388.jpg
/kaggle/working/work/testing/Degas/213312.jpg
/kaggle/working/work/testing/Degas/213263.jpg
/kaggle/working/work/testing/Degas/213493.jpg
/kaggle/working/work/testing/Degas/213278.jpg
/kaggle/working/work/testing/Degas/213641.jpg
/kaggle/working/work/testing/Degas/213738.jpg
/kaggle/working/work/testing/Degas/213579.jpg
/kaggle/working/work/testing/Degas/213648.jpg
/kaggle/working/work/testing/Degas/213616.jpg
/kaggle/working/work/testing/Degas/213742.jpg
/kaggle/working/work/testing/Degas/213593.jpg
/kaggle/working/work/testing/Degas/213513.jpg
/kaggle/working/work/testing/Degas/213675.jpg
/kaggle/working/work/testing/Degas/213555.jpg
/kaggle/working/work/testing/Degas/213685.jpg
/kaggle/working/work/testing/Degas/213581.jpg
/kaggle/working/work/testing/Degas/213

In [5]:
import numpy as np
import torch
import torchvision
from torchvision import datasets, models, transforms
import torch.utils.data as data
from torch.utils.tensorboard import SummaryWriter
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

import time, os, copy, argparse
import multiprocessing

from matplotlib import pyplot as plt

In [6]:
import torch
import torch.nn as nn

def vgg_block_single(in_ch, out_ch, kernel_size=3, padding=1):
    return nn.Sequential(
        nn.Conv2d(in_ch, out_ch, kernel_size=kernel_size, padding=padding),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
        ) 
    
def vgg_block_double(in_ch, out_ch, kernel_size=3, padding=1):
    return nn.Sequential(
        nn.Conv2d(in_ch, out_ch, kernel_size=kernel_size, padding=padding),
        nn.ReLU(),
        nn.Conv2d(out_ch, out_ch, kernel_size=kernel_size, padding=padding),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
        )


class MyVGG11(nn.Module):
    def __init__(self, in_ch, num_classes):
        super().__init__()

        self.conv_block1 =vgg_block_single(in_ch,64)
        self.conv_block2 =vgg_block_single(64,128)

        self.conv_block3 =vgg_block_double(128,256)
        self.conv_block4 =vgg_block_double(256,512)
        self.conv_block5 =vgg_block_double(512,512)

        self.fc_layers = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096), nn.ReLU(inplace=True), nn.Dropout(),
            nn.Linear(4096, 4096), nn.ReLU(inplace=True), nn.Dropout(),
            nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        
        x=self.conv_block1(x)
        x=self.conv_block2(x)

        x=self.conv_block3(x)
        x=self.conv_block4(x)
        x=self.conv_block5(x)

        x=x.view(x.size(0), -1)

        x=self.fc_layers(x)

        return x

In [7]:
train_directory = '../input/impressionist-classifier-data/training/training'
valid_directory = '../input/impressionist-classifier-data/validation/validation'

In [8]:
bs = 64 
# Number of epochs
num_epochs = 10
# Number of classes
num_classes = 10

In [9]:
from torchvision import transforms

image_transforms = { 
    'train': transforms.Compose([
        transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees=15),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
}
 
# Load data from folders
dataset = {
    'train': datasets.ImageFolder(root=train_directory, transform=image_transforms['train']),
    'valid': datasets.ImageFolder(root=valid_directory, transform=image_transforms['valid'])
}
 
# Size of train and validation data
dataset_sizes = {
    'train':len(dataset['train']),
    'valid':len(dataset['valid'])
}


In [10]:
# Create iterators for data loading
dataloaders = {
    'train':data.DataLoader(dataset['train'], batch_size=bs, shuffle=True, pin_memory=True, drop_last=True),
    'valid':data.DataLoader(dataset['valid'], batch_size=bs, shuffle=True, pin_memory=True, drop_last=True)
}

# Class names or target labels
class_names = dataset['train'].classes
print("Classes:", class_names)
 
# Print the train and validation data sizes
print("Training-set size:",dataset_sizes['train'],
      "\nValidation-set size:", dataset_sizes['valid'])

# Set default device as gpu, if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


Classes: ['Cezanne', 'Degas', 'Gauguin', 'Hassam', 'Matisse', 'Monet', 'Pissarro', 'Renoir', 'Sargent', 'VanGogh']
Training-set size: 3988 
Validation-set size: 990


In [11]:
model_ft = models.resnet18(pretrained=True)

    # Modify fc layers to match num_classes
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs,num_classes )

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/checkpoints/resnet18-5c106cde.pth


In [12]:
model_ft = model_ft.to(device)

In [13]:
pip install torchsummary

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [14]:
from torchsummary import summary
print('Model Summary:-\n')
for num, (name, param) in enumerate(model_ft.named_parameters()):
    print(num, name, param.requires_grad )
summary(model_ft, input_size=(3, 224, 224))
print(model_ft)

Model Summary:-

0 conv1.weight True
1 bn1.weight True
2 bn1.bias True
3 layer1.0.conv1.weight True
4 layer1.0.bn1.weight True
5 layer1.0.bn1.bias True
6 layer1.0.conv2.weight True
7 layer1.0.bn2.weight True
8 layer1.0.bn2.bias True
9 layer1.1.conv1.weight True
10 layer1.1.bn1.weight True
11 layer1.1.bn1.bias True
12 layer1.1.conv2.weight True
13 layer1.1.bn2.weight True
14 layer1.1.bn2.bias True
15 layer2.0.conv1.weight True
16 layer2.0.bn1.weight True
17 layer2.0.bn1.bias True
18 layer2.0.conv2.weight True
19 layer2.0.bn2.weight True
20 layer2.0.bn2.bias True
21 layer2.0.downsample.0.weight True
22 layer2.0.downsample.1.weight True
23 layer2.0.downsample.1.bias True
24 layer2.1.conv1.weight True
25 layer2.1.bn1.weight True
26 layer2.1.bn1.bias True
27 layer2.1.conv2.weight True
28 layer2.1.bn2.weight True
29 layer2.1.bn2.bias True
30 layer3.0.conv1.weight True
31 layer3.0.bn1.weight True
32 layer3.0.bn1.bias True
33 layer3.0.conv2.weight True
34 layer3.0.bn2.weight True
35 layer3.0.b

In [15]:
# Loss function
criterion = nn.CrossEntropyLoss()

# Optimizer 
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Learning rate decay
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

# Model training routine 
print("\nTraining:-\n")
def train_model(model, criterion, optimizer, scheduler, num_epochs=30):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    # Tensorboard summary
    writer = SummaryWriter()
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device, non_blocking=True)
                labels = labels.to(device, non_blocking=True)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # Record training loss and accuracy for each phase
            if phase == 'train':
                writer.add_scalar('Train/Loss', epoch_loss, epoch)
                writer.add_scalar('Train/Accuracy', epoch_acc, epoch)
                writer.flush()
            else:
                writer.add_scalar('Valid/Loss', epoch_loss, epoch)
                writer.add_scalar('Valid/Accuracy', epoch_acc, epoch)
                writer.flush()

            # deep copy the model
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model



Training:-



In [19]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=num_epochs)

Epoch 0/9
----------
train Loss: 0.3985 Acc: 0.8724
valid Loss: 0.6397 Acc: 0.7596

Epoch 1/9
----------
train Loss: 0.3991 Acc: 0.8694
valid Loss: 0.6312 Acc: 0.7687

Epoch 2/9
----------
train Loss: 0.3905 Acc: 0.8786
valid Loss: 0.6356 Acc: 0.7636

Epoch 3/9
----------
train Loss: 0.3805 Acc: 0.8837
valid Loss: 0.6278 Acc: 0.7758

Epoch 4/9
----------
train Loss: 0.3889 Acc: 0.8811
valid Loss: 0.6221 Acc: 0.7788

Epoch 5/9
----------
train Loss: 0.3669 Acc: 0.8862
valid Loss: 0.6287 Acc: 0.7727

Epoch 6/9
----------
train Loss: 0.3772 Acc: 0.8776
valid Loss: 0.6238 Acc: 0.7768

Epoch 7/9
----------
train Loss: 0.3804 Acc: 0.8811
valid Loss: 0.6272 Acc: 0.7687

Epoch 8/9
----------
train Loss: 0.3697 Acc: 0.8837
valid Loss: 0.6375 Acc: 0.7687

Epoch 9/9
----------
train Loss: 0.3792 Acc: 0.8791
valid Loss: 0.6374 Acc: 0.7667

Training complete in 30m 44s
Best val Acc: 0.778788


In [20]:
PATH="model_1.pth" 
print("\nSaving the model...")
torch.save(model_ft, PATH)


Saving the model...


# Evaluating our model

In [16]:
import numpy as np
import torch
import torchvision
from torchvision import datasets, models, transforms
import torch.utils.data as data
import multiprocessing
from sklearn.metrics import confusion_matrix

In [2]:
!git clone https://github.com/abhisingh007224/work.git

Cloning into 'work'...
remote: Enumerating objects: 775, done.
remote: Counting objects: 100% (775/775), done.
remote: Compressing objects: 100% (772/772), done.
remote: Total 775 (delta 22), reused 730 (delta 2), pack-reused 0
Receiving objects: 100% (775/775), 311.52 MiB | 43.35 MiB/s, done.
Resolving deltas: 100% (22/22), done.


In [28]:
!git clone https://github.com/abhisingh007224/model.git

Cloning into 'model'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 5 (delta 0), reused 5 (delta 0), pack-reused 0
Unpacking objects: 100% (5/5), done.


In [29]:
EVAL_MODEL='/kaggle/working/model/model/model_1.pth'
model = torch.load(EVAL_MODEL)
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [22]:
from IPython.display import FileLink
FileLink(r'./model_1.pth')

/kaggle/working/model_1.pth

In [31]:

bs = 8

EVAL_DIR='/kaggle/working/work/testing/'


# Prepare the eval data loader
eval_transform=transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])])

eval_dataset=datasets.ImageFolder(root=EVAL_DIR, transform=eval_transform)
eval_loader=data.DataLoader(eval_dataset, batch_size=bs, shuffle=True, pin_memory=True)

# Enable gpu mode, if cuda available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Number of classes and dataset-size
num_classes=len(eval_dataset.classes)
dsize=len(eval_dataset)

# Class label names
class_names=['Cezanne', 'Degas', 'Gauguin', 'Hassam', 'Matisse', 'Monet', 'Pissarro', 'Renoir', 'Sargent', 'VanGogh']

# Initialize the prediction and label lists
predlist=torch.zeros(0,dtype=torch.long, device='cpu')
lbllist=torch.zeros(0,dtype=torch.long, device='cpu')

# Evaluate the model accuracy on the dataset
correct = 0
total = 0
with torch.no_grad():
    for images, labels in eval_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        predlist=torch.cat([predlist,predicted.view(-1).cpu()])
        lbllist=torch.cat([lbllist,labels.view(-1).cpu()])

# Overall accuracy
overall_accuracy=100 * correct / total
print('Accuracy of the network on the {:d} test images: {:.2f}%'.format(dsize, 
    overall_accuracy))

# Confusion matrix
conf_mat=confusion_matrix(lbllist.numpy(), predlist.numpy())
print('Confusion Matrix')
print('-'*16)
print(conf_mat,'\n')



Accuracy of the network on the 714 test images: 80.11%
Confusion Matrix
----------------
[[54  1  4  0  4  1  3  0  4  1]
 [ 0 57  3  0  1  0  1  1  6  3]
 [ 3  0 59  0  2  1  2  3  0  2]
 [ 1  0  2 55  1  4  4  2  2  1]
 [ 1  1  6  1 57  1  1  2  1  1]
 [ 0  0  0  7  2 50  7  1  2  3]
 [ 1  0  5  1  0  3 57  3  1  1]
 [ 0  0  3  0  0  2  0 66  1  0]
 [ 0  0  0  1  2  1  2  0 62  4]
 [ 0  1  3  1  4  0  1  0  1 55]] 

